In [69]:
import re
from metagpt.actions import Action
from metagpt.logs import logger
import asyncio
from typing import Dict, List

In [70]:
def fetch_restaurant_data(restaurant_name: str) -> Dict[str, List[str]]:  # Function to fetch restaurant reviews from a data file
    # Dictionary to store restaurant reviews
    restaurant_reviews = {}

    # Open and read the restaurant-data.txt file, assuming each line contains: RestaurantName.
    with open("restaurant-data.txt", "r") as file:
        for line in file:
            line = line.strip()
            if not line:
                continue  # Skip empty lines
            try:
                # Split each line into restaurant name and review
                name, review = line.split(". ", 1)
                name = name.strip()
                review = review.strip()
                # Store review in the dictionary under the corresponding restaurant name
                if name not in restaurant_reviews:
                    restaurant_reviews[name] = []
                restaurant_reviews[name].append(review)
            except ValueError:
                # Skip lines that do not match the expected format
                continue

    # Find reviews for the specified restaurant (case insensitive search)
    reviews = []
    for key in restaurant_reviews.keys():
        if restaurant_name.lower() in key.lower():
            reviews.extend(restaurant_reviews[key])

    # Return the reviews if found, otherwise return a "No reviews found" message
    return {restaurant_name: reviews} if reviews else {restaurant_name: ["No reviews found for this restaurant."]}

In [64]:
class Extract(Action):
    PROMPT_TEMPLATE: str = """
    Extract the name of the restaurant from ```{query}```. 
    Return the extracted name following this output format ```the name is: XXX``` with NO other texts.
    """

    name: str = "ExtractName"

    async def run(self, query: str):
        prompt = self.PROMPT_TEMPLATE.format(query=query)
        
        rsp = await self._aask(prompt)

        extracted_name = Extract.extract_name(rsp)
    
        return extracted_name

    @staticmethod
    def extract_name(rsp):
        pattern = r"the name is: (.*)"
        match = re.search(pattern, rsp)
        name = match.group(1) if match else rsp
        return name.strip()
    
class Compare(Action):
    PROMPT_TEMPLATE: str = """
    Consider the following list of names:\n
    ```
    {names}
    ```\n
    Select the entry which is most simmilar to ```{query}```.
    Return the selected entry following this output format ```the entry is: XXX``` with NO other texts.
    """

    name: str = "CompareNames"

    async def run(self, query: str, names: str):
        prompt = self.PROMPT_TEMPLATE.format(query=query, names=names)
        
        rsp = await self._aask(prompt)

        extracted_name = Compare.compare_names(rsp)
    
        return extracted_name

    @staticmethod
    def compare_names(rsp):
        pattern = r"the entry is: (.*)"
        match = re.search(pattern, rsp)
        name = match.group(1) if match else rsp
        return name.strip()

In [76]:
import nest_asyncio
nest_asyncio.apply()

names = """
"McDonald's",
"Subway",
"Taco Bell",
"Chick-fil-A",
"Applebee's",
"Olive Garden",
"Cheesecake Factory",
"Buffalo Wild Wings",
"Starbucks",
"Krispy Kreme",
"Panera Bread",
"Tim Horton's",
"Chipotle",
"In-n-Out",
"Five Guys",
"Panda Express",
"Pret A Manger",
"Cinnabon",
"IHOP",
"Burger King"
"""

query="What is the overall score forChipotleee?"

role = Extract()
logger.info(query)
extracted_name = await role.run(query)

print(extracted_name)

role = Compare()
logger.info(query)
extracted_name = await role.run(extracted_name, names)

print(extracted_name)


2024-11-07 16:18:23.690 | INFO     | __main__:<module>:30 - What is the overall score forChipotleee?


the name is: Chipotle

2024-11-07 16:18:26.440 | INFO     | metagpt.utils.cost_manager:update_cost:108 - prompt_tokens: 69, completion_tokens: 8
2024-11-07 16:18:26.441 | INFO     | __main__:<module>:36 - What is the overall score forChipotleee?


ee
Chipotleee
the entry is: Chipotle

2024-11-07 16:18:33.126 | INFO     | metagpt.utils.cost_manager:update_cost:108 - prompt_tokens: 176, completion_tokens: 7



Chipotle


In [77]:
reviews = fetch_restaurant_data(extracted_name).get(extracted_name)

reviews

['Chipotle offers good, fresh Mexican-inspired fare with customizable options. The customer service is average, with long lines but generally efficient staff.',
 'The burritos were enjoyable with fresh ingredients. The service was average, with some staff seeming disinterested in customer interactions.',
 'The food at Chipotle is consistently good, with fresh ingredients and customizable options. The customer service was average, with long lines during peak hours.',
 "Chipotle's food is enjoyable, with fresh ingredients and customizable options. The service is average, with occasional long lines during peak hours.",
 'The food was good, but the service was average. The burrito bowl was satisfying and flavorful, though the line moved slowly due to understaffing.',
 "Chipotle's food is good, with fresh ingredients and customizable options. The service is average, with occasional long lines during peak hours.",
 "Chipotle's build-your-own concept results in satisfying meals with fresh ing